## Import Library

In [3]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from geopy.distance import geodesic
import folium
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics import pairwise_distances
from collections import defaultdict
from sklearn.preprocessing import StandardScaler

import json
import os

import requests

In [1]:
# parameter clustering
n_cluster = 45
harga_km = 1 # belum terpakai
max_cluster = 60
tonase_truk = 8 # terpakai untuk menghitung estimasi pengiriman

units = 'm'
profile = 'driving-hgv'

# variabel input
kios_csv = 'kios_aws.csv'
desa_csv = 'desa_aws_8264.csv'
dc_csv = 'dc.csv'

# variabel output
folder_name = f"{n_cluster}_cluster"
desa = 'desa'
output_desa_df = f'{desa}_{n_cluster}.xlsx'
output_desa_DC = f'dc_{desa}_{n_cluster}.xlsx'
output_desa_map = f'map_{desa}_{n_cluster}.html'

kios = 'kios'
output_kios_df = f'{kios}_{n_cluster}.xlsx'
output_kios_DC = f'dc_{kios}_{n_cluster}.xlsx'
output_kios_map = f'map_{kios}_{n_cluster}.html'

dist_file_name = f"distance_list_{n_cluster}.json"
duration_file_name = f"duration_list_{n_cluster}.json"


In [2]:
def elbow_and_silhouete(csv_file, optimal_clusters=20, harga_km=1, tonase_truk=4):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    df['latitude'] = df['lat_customer']
    df['longitude'] = df['lon_customer']
    
    # # Filter the DataFrame based on latitude and longitude
    # df = df[(df['latitude'] < -6.4) & (df['latitude'] > -8) & (df['longitude'] < 112) & (df['longitude'] > 108.57)]

    # Fill NaN values in 'Alokasi_per_bulan'
    df['alokasi_per_bulan'] = df['alokasi_per_bulan'].fillna(0)
    
    # Get coordinates for clustering
    coords = df[['latitude', 'longitude', 'alokasi_per_bulan']].values
    
    # For Elbow Analysis
    inertia_values = []
    
    # For Silhouette Analysis
    silhouette_values = []
    
    cluster_range = range(2, optimal_clusters + 1)
    
    for k in cluster_range:
        # distance_matrix = pairwise_distances(coords, metric='euclidean') #pakai distance matrix
        kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
        kmeans.fit(coords)
        
        # Append the inertia (sum of squared distances) for elbow analysis
        inertia_values.append(kmeans.inertia_)
        
        # Calculate and append silhouette score (only if more than 1 cluster)
        if k > 1:  # Silhouette score is undefined for 1 cluster
            silhouette_avg = silhouette_score(coords, kmeans.labels_)
            silhouette_values.append(silhouette_avg)
        else:
            silhouette_values.append(-1)  # Placeholder for silhouette score when k = 1

    # Plotting Elbow Method
    plt.figure(figsize=(12, 6))

    # Elbow Method Plot
    plt.subplot(1, 2, 1)
    plt.plot(cluster_range, inertia_values, marker='o')
    plt.title('Elbow Method')
    plt.xlabel('Number of clusters')
    plt.ylabel('Inertia')
    plt.grid()

    # Silhouette Analysis Plot
    plt.subplot(1, 2, 2)
    plt.plot(cluster_range[1:], silhouette_values[1:], marker='o')  # Exclude k=1 for silhouette plot
    plt.title('Silhouette Analysis')
    plt.xlabel('Number of clusters')
    plt.ylabel('Silhouette Score')
    plt.grid()

    plt.tight_layout()
    plt.show()

Memperoleh Distance Menggunakan ORS API "Matrix", mengembalikan file json dalam variable file_name

In [4]:
def get_distance(df, centroids, distance_file_name):
    coord_by_dc = defaultdict(list)

    # Load data dari file jika file ada, atau inisialisasi test_list baru
    if os.path.exists(distance_file_name):
        with open(distance_file_name, 'r') as file:
            distance_list = json.load(file)
    else:
        distance_list = []
    
    distance_values = []

    # Iterasi melalui setiap baris data dan masukkan koordinat kios berdasarkan cluster ke dalam dictionary
    for _, row in df.iterrows():
        label = row['nearest_DC']
        lat = float(row['latitude'])
        lon = float(row['longitude'])
        # Tambahkan koordinat centroid dan koordinat kios ke dalam list untuk label cluster yang sesuai
        coord_by_dc[label].append([lon, lat])

    # Tambahkan centroid ke dalam dictionary
    for i, centroid in enumerate(centroids):
        coord_by_dc[i].insert(0, [float(centroid[1]), float(centroid[0])])  # Menambahkan koordinat centroid (long, lat)

    # Ubah defaultdict menjadi dictionary biasa jika diinginkan
    coord_by_dc = dict(coord_by_dc)

    # pengecekan hasil kalkulasi jarak sebelumnya dalam distance_list.json
    existing_centroid = {entry['DC'] for entry in distance_list}

    # memperoleh distance_matrix menggunakan ORS
    for centroid_index, locations in coord_by_dc.items():
        centroid_index = f'DC{centroid_index+1}'
        if centroid_index in existing_centroid:
            print(f"Distance cluster {centroid_index} already exists!")
            continue
        else:
            # Ambil daftar koordinat village untuk centroid ini
            print(f"Calculating distance for centroid {centroid_index}...")

            village_coords = [(loc[0], loc[1]) for loc in locations]

            # Buat payload untuk request
            request = {
                'locations': village_coords,
                'profile': profile,
                'metrics': ['distance'],
                'units': units
            }

            headers = {
                'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
                'Authorization': '',  # Tambahkan token Anda di sini
                'Content-Type': 'application/json; charset=utf-8'
            }

            url = 'https://ds-route-service-api.pupuk.in/ors/v2/matrix/driving-hgv'

            # Mengirim permintaan dan menangani respons
            try:
                response = requests.post(url=url, headers=headers, json=request)
                response.raise_for_status()  # Raise error jika status code bukan 2xx

                # Parsing hasil
                village_matrix = response.json()
                distances = village_matrix.get('distances', [])
                distances = [distance if distance is not None else 0 for distance in distances]

                # Simpan hasil untuk centroid ini dalam list
                distance_list.append({
                    'DC': centroid_index,
                    'distances': distances
                })
                print(f"Processed centroid {centroid_index}: {len(distances)} distances successfully calculated.")

            except requests.exceptions.RequestException as e:
                response = response.json()
                print(f"Error while processing centroid {centroid_index}:\nCode: {response['error']['code']}\nMessage: {response['error']['message']}")
                distance_list.append({
                    'DC': centroid_index,
                    'distances': [[0] * len(village_coords)]
                })
            except KeyError as e:
                print(f"Response format error for centroid {centroid_index}. Missing key:", e)
                distance_list.append({
                    'DC': centroid_index,
                    'distances': [[0] * len(village_coords)]
                })
            except Exception as e:
                print(f"Unexpected error for centroid {centroid_index}:", e)
                distance_list.append({
                    'DC': centroid_index,
                    'distances': [[0] * len(village_coords)]
                })

    # Mengambil jarak dari DC ke konsumen lain yang diambil dari index 0, kemudian mengembalikan nilai tersebut ke dalam distance_value
    for _, value in enumerate(distance_list):
        distances = value['distances']
        distance_values.extend(distances[0][1:]) 

    distance_values = [distance / 1000 if distance is not None else 0 for distance in distance_values]
   
    with open(distance_file_name, 'w') as file:
        json.dump(distance_list, file)

    # Output semua hasil jarak
    print("Distance calculations completed for all centroids.")

    return distance_values

In [5]:
def get_duration(df, centroids, duration_file_name):
    coord_by_dc = defaultdict(list)

    # Load data dari file jika file ada, atau inisialisasi test_list baru
    if os.path.exists(duration_file_name):
        with open(duration_file_name, 'r') as file:
            duration_list = json.load(file)
    else:
        duration_list = []
    
    duration_values = []

    # Iterasi melalui setiap baris data dan masukkan koordinat kios berdasarkan cluster ke dalam dictionary
    for _, row in df.iterrows():
        label = row['nearest_DC']
        lat = float(row['latitude'])
        lon = float(row['longitude'])
        # Tambahkan koordinat centroid dan koordinat kios ke dalam list untuk label cluster yang sesuai
        coord_by_dc[label].append([lon, lat])

    # Tambahkan centroid ke dalam dictionary
    for i, centroid in enumerate(centroids):
        coord_by_dc[i].insert(0, [float(centroid[1]), float(centroid[0])])  # Menambahkan koordinat centroid (long, lat)

    # Ubah defaultdict menjadi dictionary biasa jika diinginkan
    coord_by_dc = dict(coord_by_dc)

    # pengecekan hasil kalkulasi jarak sebelumnya dalam duration_list.json
    existing_centroid = {entry['DC'] for entry in duration_list}

    # memperoleh duration_matrix menggunakan ORS
    for centroid_index, locations in coord_by_dc.items():
        centroid_index = f'DC{centroid_index+1}'
        if centroid_index in existing_centroid:
            print(f"Duration cluster {centroid_index} already exists!")
            continue
        else:
            # Ambil daftar koordinat village untuk centroid ini
            print(f"Calculating duration for centroid {centroid_index}...")

            village_coords = [(loc[0], loc[1]) for loc in locations]

            # Buat payload untuk request
            request = {
                'locations': village_coords,
                'profile': 'driving-hgv',
                'metrics': ['duration'],
            }

            headers = {
                'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
                'Authorization': '',  # Tambahkan token Anda di sini
                'Content-Type': 'application/json; charset=utf-8'
            }

            url = 'https://ds-route-service-api.pupuk.in/ors/v2/matrix/driving-hgv'

            # Mengirim permintaan dan menangani respons
            try:
                response = requests.post(url=url, headers=headers, json=request)
                response.raise_for_status()  # Raise error jika status code bukan 2xx

                # Parsing hasil
                village_matrix = response.json()
                durations = village_matrix.get('durations', [])
                durations = [duration if duration is not None else 0 for duration in durations]
                # Simpan hasil untuk centroid ini dalam list
                duration_list.append({
                    'DC': centroid_index,
                    'durations': durations
                })
                print(f"Processed centroid {centroid_index}: {len(durations)} durations successfully calculated.")

            except requests.exceptions.RequestException as e:
                response = response.json()
                print(f"Error while processing centroid DC{centroid_index}:\nCode: {response['error']['code']}\nMessage: {response['error']['message']}")
                duration_list.append({
                    'DC': centroid_index,
                    'durations': [[0] * len(village_coords)]
                })
            except KeyError as e:
                print(f"Response format error for centroid DC{centroid_index}. Missing key:", e)
                duration_list.append({
                    'DC': centroid_index,
                    'durations': [[0] * len(village_coords)]
                })
            except Exception as e:
                print(f"Unexpected error for centroid DC{centroid_index}:", e)
                duration_list.append({
                    'DC': centroid_index,
                    'durations': [[0] * len(village_coords)]
                })

    # Mengambil jarak dari DC ke konsumen lain yang diambil dari index 0, kemudian mengembalikan nilai tersebut ke dalam duration_value
    for _, value in enumerate(duration_list):
        durations = value['durations']
        duration_values.extend(durations[0][1:]) 

    duration_values = [duration / 60 if duration is not None else 0 for duration in duration_values]

    with open(duration_file_name, 'w') as file:
        json.dump(duration_list, file)

    # Output semua hasil jarak
    print("duration calculations completed for all centroids.")

    return duration_values

In [6]:
def determine_optimal_DCs(csv_file, optimal_clusters=20, harga_km=1, tonase_truk=4, output_DC='optimal_DC_locations_euc.csv', output_df='desa_with_DC_assignments_euc.csv', dist_file_name ='dist_file_name_def.json', duration_file_name ='dist_file_name_def.json'):
    df = pd.read_csv(csv_file)
    df['latitude'] = df['lat_customers']
    df['longitude'] = df['lon_customers']
    # df = df[(df['latitude'] < -6.4) & (df['latitude'] > -8) & (df['longitude'] < 112) & (df['longitude'] > 108.57)]
    df['alokasi_per_bulan'] = df['alokasi_per_bulan'].fillna(1)

    df_weighted = df.loc[df.index.repeat(df['alokasi_per_bulan'].astype(int))]
    
    # Get coordinates for clustering
    X = df_weighted[['latitude', 'longitude']].values
    kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
    kmeans.fit(X)

    centroids = kmeans.cluster_centers_
    
    df['nearest_DC'] = kmeans.predict(df[['latitude', 'longitude']].values)
    df.sort_values(by='nearest_DC',ascending=True,inplace=True)
    
    DC_df = pd.DataFrame(centroids, columns=['DC_latitude', 'DC_longitude'])
    DC_df['DC_name'] = [f"DC{i+1}" for i in range(len(DC_df))]

    def find_DC_details(coords, label):
        DC_coords = centroids[label]
        DC_name = DC_df.loc[label, 'DC_name']
        return DC_name, DC_coords[0], DC_coords[1]


    df[['assigned_DC', 'DC_latitude', 'DC_longitude']] = df.apply(
        lambda row: find_DC_details((row['latitude'], row['longitude']), row['nearest_DC']),
        axis=1,
        result_type='expand'
    )

    df['estimasi_pengiriman'] = df.apply(
        lambda row: math.ceil(row['alokasi_per_bulan'] / tonase_truk), axis=1
    )

    df['distance_to_DC_km'] = get_distance(df, centroids, dist_file_name)
    df['durations_in_min'] = get_duration(df, centroids, duration_file_name)
    
    # df['estimasi_harga'] = df.apply(
    #     lambda row: math.ceil(row['alokasi_per_bulan'] / tonase_truk) * harga_km  * math.ceil(row['distance_to_DC_km']), axis=1
    # )
    
    DC_estimasi_jumlah_pengiriman = df.groupby('nearest_DC').agg(
        estimasi_pengiriman=("estimasi_pengiriman","sum"),
        mean_dist_to_endcust=('distance_to_DC_km','mean'),
        mean_urea_DC_bulan=("urea_per_bulan",'mean'),
        mean_NPK_DC_bulan=("NPK_per_bulan",'mean'),
        kios=("Kios","mean"),
        jumlah_desa=("nearest_DC", "size"),
        jumlah_alokasi_DC_bulan=("alokasi_per_bulan","sum"),
        mean_alokasi_DC_bulan=("alokasi_per_bulan","mean"),
        jumlah_real_penjualan_DC_bulan=("realisasi_penjualan_desa_per_bulan","sum"),
        jumlah_transaksi_DC_bulan=("jumlah_transaksi_desa_per_bulan","sum"),   
        )
        
    DC_df = DC_df.merge(DC_estimasi_jumlah_pengiriman, left_index=True, right_on='nearest_DC', how='left')

    df = df.drop(columns=['nearest_DC'])

    # Save the desa data with assigned DCs to a new CSV file
    df.to_excel(output_df, index=False, sheet_name='all_dc')
    DC_df.to_excel(output_DC, index=False)

    return df, DC_df

In [64]:
# df = pd.read_csv('desa_aws_8264.csv')
# df['latitude'] = df['lat_customer']
# df['longitude'] = df['lon_customer']

# df['alokasi_per_bulan'] = df['alokasi_per_bulan'].fillna(1)
# coords = df[['latitude', 'longitude']].values

# df_weighted = df.loc[df.index.repeat(df['alokasi_per_bulan'].astype(int))]

# df
# # Get coordinates for clustering based on alokasi
# coords = df_weighted[['latitude', 'longitude']].values
# kmeans = KMeans(n_clusters=45, random_state=42)
# kmeans.fit(coords)

# centroids = kmeans.cluster_centers_

# df['nearest_DC'] = kmeans.predict(df[['latitude', 'longitude']].values)

# DC_df = pd.DataFrame(centroids, columns=['DC_latitude', 'DC_longitude'])
# DC_df['DC_name'] = [f"DC{i+1}" for i in range(len(DC_df))]

# # def find_DC_details(desa_coords, label):
# #     DC_coords = centroids[label]
# #     distance_to_DC = geodesic(desa_coords, DC_coords).kilometers
# #     DC_name = DC_df.loc[label, 'DC_name']
# #     return DC_name, DC_coords[0], DC_coords[1], distance_to_DC

# def find_DC_details(coords, label):
#     DC_coords = centroids[label]
#     DC_name = DC_df.loc[label, 'DC_name']
#     return DC_name, DC_coords[0], DC_coords[1]


# df[['assigned_DC', 'DC_latitude', 'DC_longitude']] = df.apply(
#     lambda row: find_DC_details((row['latitude'], row['longitude']), row['nearest_DC']),
#     axis=1,
#     result_type='expand'
# )

# df['estimasi_pengiriman'] = df.apply(
#     lambda row: math.ceil(row['alokasi_per_bulan'] / 8), axis=1
# )

# df.sort_values(by='nearest_DC',ascending=True,inplace=True)
# df

# # # df['estimasi_harga'] = df.apply(
# # #     lambda row: math.ceil(row['alokasi_per_bulan'] / tonase_truk) * harga_km  * math.ceil(row['distance_to_DC_km']), axis=1
# # # )

# # # # Sum estimasi_harga for each DC
# # # DC_estimasi_harga = df.groupby('nearest_DC')['estimasi_harga'].sum().reset_index()

# # # # Merge total estimasi_harga with DC_df
# # # DC_df = DC_df.merge(DC_estimasi_harga, left_index=True, right_on='nearest_DC', how='left')
# # # DC_df = DC_df.rename(columns={'estimasi_harga': 'total_estimasi_harga'})

# # DC_estimasi_jumlah_pengiriman = df.groupby('nearest_DC')['estimasi_pengiriman'].sum().reset_index()
# # DC_df = DC_df.merge(DC_estimasi_jumlah_pengiriman, left_index=True, right_on='nearest_DC', how='left')

# # df = df.drop(columns=['nearest_DC'])

# # DC_estimasi_jumlah_pengiriman = df.groupby('nearest_DC').agg(
# #     est_pengiriman=("estimasi_pengiriman","sum"),
# #     # mean_dist_to_endcust=('distance_to_DC_km','mean'),
# #     mean_urea_DC_bulan=("urea_per_bulan",'mean'),
# #     mean_NPK_DC_bulan=("NPK_per_bulan",'mean'),
# #     kios=("Kios","count"),
# #     jumlah_alokasi_DC_bulan=("alokasi_per_bulan","sum"),
# #     jumlah_real_penjualan_DC_bulan=("realisasi_penjualan_kios_per_bulan","sum"),
# #     jumlah_transaksi_DC_bulan=("jumlah_transaksi_kios_per_bulan","sum"),   
# # )
# # DC_df = DC_df.merge(DC_estimasi_jumlah_pengiriman, left_index=True, right_on='nearest_DC', how='left')
# # DC_df

In [65]:
# coord_by_dc = defaultdict(list)

# distance_list = []

# # Iterasi melalui setiap baris data dan masukkan koordinat kios berdasarkan cluster ke dalam dictionary
# for _, row in df.iterrows():
#     label = row['nearest_DC']
#     lat = float(row['latitude'])
#     lon = float(row['longitude'])
#     # Tambahkan koordinat centroid dan koordinat kios ke dalam list untuk label cluster yang sesuai
#     coord_by_dc[label].append([lon, lat])

# # Tambahkan centroid ke dalam dictionary
# for i, centroid in enumerate(centroids):
#     coord_by_dc[i].insert(0, [float(centroid[1]), float(centroid[0])])  # Menambahkan koordinat centroid (long, lat)

# # Ubah defaultdict menjadi dictionary biasa jika diinginkan
# coord_by_dc = dict(coord_by_dc)

In [66]:
# coord_by_dc

In [67]:
# # checking salah satu cluster yang bermasalah

# coord_by_dc_23 = coord_by_dc[30]
# # village_coords = [(loc[0][0], loc[0][1]) for loc in coord_by_dc.items()]

# # Buat payload untuk request
# request = {
#     'locations': coord_by_dc_23,
#     'profile': 'driving-hgv',
#     'metrics': ['distance'],
#     'units': 'm'
# }

# headers = {
#     'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
#     'Authorization': '',  # Tambahkan token Anda di sini
#     'Content-Type': 'application/json; charset=utf-8'
# }

# url = 'https://ds-route-service-api.pupuk.in/ors/v2/matrix/driving-hgv'

# # Mengirim permintaan dan menangani respons
# try:
#     response = requests.post(url=url, headers=headers, json=request)
#     response.raise_for_status()  # Raise error jika status code bukan 2xx

#     # Parsing hasil
#     village_matrix = response.json()
#     distances = village_matrix.get('distances', [])

#     print(f"Processed centroid 1: {len(distances)} distances calculated.")

# except requests.exceptions.RequestException as e:
#     response = response.json()
#     print(f"Error while processing centroid 1:\nCode: {response['error']['code']}\nMessage: {response['error']['message']}")

# except KeyError as e:
#     print(f"Response format error for centroid 23. Missing key:", e)

# except Exception as e:
#     print(f"Unexpected error for centroid 23:", e)

In [68]:
# coord_by_dc_24 = coord_by_dc[24]
# coord_by_dc_20 = coord_by_dc[20]
# coord_by_dc_23_latlong = [[lat, long] for long, lat in coord_by_dc_23]
# coord_by_dc_24_latlong = [[lat, long] for long, lat in coord_by_dc_24]
# coord_by_dc_20_latlong = [[lat, long] for long, lat in coord_by_dc_20]

# m = folium.Map(zoom_start=5)

# for coords in coord_by_dc_23_latlong:
#     folium.Marker(
#         location=coords,
#         icon=folium.Icon(color='green', icon='industry', prefix='fa')
#     ).add_to(m)

# for coords in coord_by_dc_24_latlong:
#     folium.Marker(
#         location=coords,
#         icon=folium.Icon(color='blue', icon='industry', prefix='fa')
#     ).add_to(m)

# for coords in coord_by_dc_20_latlong:
#     folium.Marker(
#         location=coords,
#         icon=folium.Icon(color='red', icon='industry', prefix='fa')
#     ).add_to(m)
# m

In [69]:
# # Inisialisasi list untuk menyimpan hasil
# distance_list = []

# # Iterasi melalui dictionary coord_by_dc
# for centroid_index, locations in coord_by_dc.items():
#     # Ambil daftar koordinat village untuk centroid ini
#     village_coords = [(loc[0], loc[1]) for loc in locations]

#     # Buat payload untuk request
#     request = {
#         'locations': village_coords,
#         'profile': 'driving-hgv',
#         'metrics': ['distance'],
#         'units': 'm'
#     }

#     headers = {
#         'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
#         'Authorization': '',  # Tambahkan token Anda di sini
#         'Content-Type': 'application/json; charset=utf-8'
#     }

#     url = f'http://34.34.220.116:8082/ors/v2/matrix/driving-hgv'

#     # Mengirim permintaan dan menangani respons
#     try:
#         response = requests.post(url=url, headers=headers, json=request)
#         response.raise_for_status()  # Raise error jika status code bukan 2xx

#         # Parsing hasil
#         village_matrix = response.json()
#         distances = village_matrix.get('distances', [])

#         # Simpan hasil untuk centroid ini dalam list
#         distance_list.append({
#             'centroid_index': centroid_index,
#             'distances': distances
#         })
#         print(f"Processed centroid {centroid_index}: {len(distances)} distances calculated.")

#     except requests.exceptions.RequestException as e:
#         response = response.json()
#         print(f"Error while processing centroid 23:\nCode: {response['error']['code']}\nMessage: {response['error']['message']}")
#         distance_list.append({
#             'centroid_index': centroid_index,
#             'distances': [None] * len(village_coords)  # Mengembalikan None jika ada error
#         })

#     except KeyError as e:
#         print(f"Response format error for centroid {centroid_index}. Missing key:", e)
#         distance_list.append({
#             'centroid_index': centroid_index,
#             'distances': [None] * len(village_coords)  # Mengembalikan None jika ada error
#         })
#     except Exception as e:
#         distance_list.append({
#             'centroid_index': centroid_index,
#             'distances': [None] * len(village_coords)  # Mengembalikan None jika ada error
#         })
#         print(f"Unexpected error for centroid {centroid_index}:", e)

# # Output semua hasil jarak
# print("Distance calculations completed for all centroids.")

In [70]:
# target_centroid_index = 23
# for item in distance_list:
#     if item['centroid_index'] == target_centroid_index:
#         # Jika ditemukan, lakukan sesuatu, misalnya print data
#         print(item)
#         break  # Me

In [71]:
# df['distance_to_DC_km'] = None  # Jika kolom 'distance_to_DC_km' belum ada
# # Buat list kosong untuk menampung hasil
# distance_values = []

# # Iterasi melalui distance_list dan simpan dalam list
# for idx, value in enumerate(distance_list):
#     distances = value['distances']
#     # Menambahkan elemen yang relevan (distances[0][1:])
#     distance_values.extend(distances[0][1:])  # Simpan hasilnya di list

# distance_values = [distance / 1000 if distance is not None else None for distance in distance_values]

# df['distance_to_DC_km'] = distance_values
# df

In [7]:
def visualize_desa_DC_map(desa_file, DC_file, output_file, folder_name):
    read_main = os.path.join(folder_name, desa_file)
    read_dc = os.path.join(folder_name,DC_file)
    desa_df = pd.read_excel(read_main, engine='openpyxl')
    DC_df = pd.read_excel(read_dc, engine='openpyxl')
    
    map_center = [desa_df['latitude'].mean(), desa_df['longitude'].mean()]
    
    m = folium.Map(location=map_center, zoom_start=10)

    for _, row in DC_df.iterrows():
        folium.Marker(
            location=[row['DC_latitude'], row['DC_longitude']],
            tooltip=f"""
            DC: {row['DC_name']}<br>
            Pengiriman: {row['estimasi_pengiriman']}<br>
            Rata-Rata Jarak: {row['mean_dist_to_endcust']}<br>
            jumlah alokasi: {row['jumlah_alokasi_DC_bulan']}<br>
            Urea (ton): {row['mean_urea_DC_bulan']}<br>
            NPK (ton): {row['mean_NPK_DC_bulan']}<br>
            kios: {row['kios']}<br>
            Realisasi Penjualan: {row['jumlah_real_penjualan_DC_bulan']}<br>
            Jumlah Transaksi: {row['jumlah_transaksi_DC_bulan']}<br>""",
            icon=folium.Icon(color='green', icon='industry', prefix='fa')
        ).add_to(m)

    for _, row in desa_df.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=3,
            color='#ff8b3d',
            fill=True,
            fill_color='#ff8b3d',
            fill_opacity=0.6,
            tooltip=f"Desa: {row['customers']}<br>Assigned DC: {row['assigned_DC']}<br>Distance: {row['distance_to_DC_km']:.2f} km<br>Pengiriman: {row['estimasi_pengiriman']}"
        ).add_to(m)

        folium.PolyLine(
            locations=[[row['latitude'], row['longitude']], [row['DC_latitude'], row['DC_longitude']]],
            color='gray',
            weight=1,
            opacity=0.7
        ).add_to(m)
    
    save_file = os.path.join(folder_name, output_file)
    m.save(save_file)

In [8]:
def visualize_kios_DC_map(kios_file, DC_file, output_file):
    kios_df = pd.read_csv(kios_file)
    # DC_df = pd.read_excel(DC_file,engine='openpyxl')
    DC_df = pd.read_csv(DC_file)
    
    print(DC_df[['DC_latitude', 'DC_longitude']].isna().sum())
    print(kios_df[['latitude', 'longitude']].isna().sum())
    map_center = [kios_df['latitude'].mean(), kios_df['longitude'].mean()]
    
    m = folium.Map(location=map_center, zoom_start=10)

    for _, row in DC_df.iterrows():
        folium.Marker(
            location=[row['DC_latitude'], row['DC_longitude']],
            tooltip=f"""
            DC: {row['DC_name']}<br>
            Pengiriman: {row['est_pengiriman']}<br>
            Rata-Rata Jarak: {row['mean_dist_to_endcust']}<br>
            Urea (ton): {row['mean_urea_DC_bulan']}<br>
            NPK (ton): {row['mean_NPK_DC_bulan']}<br>
            kios: {row['kios']}<br>
            jumlah alokasi: {row['jumlah_alokasi_DC_bulan']}<br>
            Realisasi Penjualan: {row['jumlah_real_penjualan_DC_bulan']}<br>
            Jumlah Transaksi: {row['jumlah_transaksi_DC_bulan']}<br>""",
            icon=folium.Icon(color='green', icon='industry', prefix='fa')
        ).add_to(m)

    for _, row in kios_df.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=3,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6,
            tooltip=f"Kios: {row['Nama_Kios']}<br>Assigned DC: {row['assigned_DC']}<br>Distance: {row['distance_to_DC_km']:.2f} km<br>Pengiriman: {row['estimasi_pengiriman']}"
        ).add_to(m)

        folium.PolyLine(
            locations=[[row['latitude'], row['longitude']], [row['DC_latitude'], row['DC_longitude']]],
            color='gray',
            weight=1,
            opacity=0.7
        ).add_to(m)

    m.save(output_file)

### Determine Clustering For DESA

In [ ]:
# # Membuat folder jika belum ada
# if not os.path.exists(folder_name):
#     os.makedirs(folder_name)
#     print(f"Folder '{folder_name}' berhasil dibuat!")
# else:
#     print(f"Folder '{folder_name}' sudah ada.")

In [18]:
# # elbow dan silhouetee kios
# elbow_and_silhouete(
#     csv_file=desa_csv,
#     optimal_clusters=max_cluster,
#     harga_km=harga_km, # belum terpakai
#     tonase_truk=tonase_truk # belum terpakai
# )

In [9]:
# memperoleh hasil clustering pada data desa
cust_locations, DC_locations = determine_optimal_DCs(
    csv_file=desa_csv,
    optimal_clusters=n_cluster,
    harga_km=harga_km,
    tonase_truk=tonase_truk,
    output_df=output_desa_df,
    output_DC=output_desa_DC,
    dist_file_name=dist_file_name,
    duration_file_name=duration_file_name,
    )

c:\Users\DELL\anaconda3\envs\fastapi\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\DELL\anaconda3\envs\fastapi\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


Calculating distance for centroid DC1...
Processed centroid DC1: 368 distances successfully calculated.
Calculating distance for centroid DC2...
Processed centroid DC2: 114 distances successfully calculated.
Calculating distance for centroid DC3...
Processed centroid DC3: 134 distances successfully calculated.
Calculating distance for centroid DC4...
Processed centroid DC4: 72 distances successfully calculated.
Calculating distance for centroid DC5...
Processed centroid DC5: 190 distances successfully calculated.
Calculating distance for centroid DC6...
Processed centroid DC6: 130 distances successfully calculated.
Calculating distance for centroid DC7...
Processed centroid DC7: 73 distances successfully calculated.
Calculating distance for centroid DC8...
Processed centroid DC8: 83 distances successfully calculated.
Calculating distance for centroid DC9...
Processed centroid DC9: 232 distances successfully calculated.
Calculating distance for centroid DC10...
Processed centroid DC10: 

In [ ]:
# visualize_kios_DC_map(DC_file='dc_kios_45.csv',kios_file='kios_45.csv',output_file='map_desa_45.html')

In [120]:
visualize_desa_DC_map(desa_file=output_desa_df, DC_file=output_desa_DC, output_file=output_desa_map, folder_name=folder_name)